In [3]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader, Subset

# import numpy as np


import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Data preprocessing and loading
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,download=False, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,download=False, transform=transform)
train_subset = Subset(train_dataset, range(200))
test_subset = Subset(test_dataset, range(50))
train_loader = DataLoader(train_subset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=10, shuffle=False)

In [5]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_block1 = nn.Sequential(nn.Conv2d(3, 32, kernel_size=3, padding=1),nn.ReLU(),nn.MaxPool2d(2))
        self.conv_block2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),nn.MaxPool2d(2))
        self.dense1 = nn.Linear(64 * 8 * 8, 512)
        self.dense2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        return x

In [6]:
class CNNWithBNDropout(nn.Module):
    def __init__(self):
        super(CNNWithBNDropout, self).__init__()
        self.conv_block1 = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        self.conv_block2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        self.dense1 = nn.Linear(64 * 8 * 8, 512)
        self.dense2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.dropout(x)
        return x

In [8]:
model1 = SimpleCNN()
model2 = CNNWithBNDropout()

criterion = nn.CrossEntropyLoss()

optimizer1 = optim.Adam(model1.parameters(), lr=0.001)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)

In [11]:
def train(model, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
    for data, target in train_loader:
        output = model(data)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        predicted = torch.argmax(output.data, dim=1)
        total_train += target.size(0)
        correct_train += (predicted == target).sum().item()

    avg_train_loss = train_loss / len(train_loader)
    train_acc = 100 * correct_train / total_train
    model.eval()
    test_loss = 0.0
    correct_test = 0
    total_test = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            predicted = torch.argmax(output.data, dim=1)
            total_test += target.size(0)
            correct_test += (predicted==target).sum().item()
            avg_test_loss = test_loss/len(test_loader)
            test_acc = 100 * correct_test/total_test
            print(f'Epoch: [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f},rain Accuracy: {train_acc:.4f}%, Test Loss: {avg_test_loss:.4f}, TestAccuracy: {test_acc:.4f}%')


train(model1, optimizer1, criterion, 20)
train(model2, optimizer2, criterion, 30)

Epoch: [20/20], Train Loss: 2.2554,rain Accuracy: 13.5000%, Test Loss: 0.4482, TestAccuracy: 10.0000%
Epoch: [20/20], Train Loss: 2.2554,rain Accuracy: 13.5000%, Test Loss: 0.8963, TestAccuracy: 15.0000%
Epoch: [20/20], Train Loss: 2.2554,rain Accuracy: 13.5000%, Test Loss: 1.3558, TestAccuracy: 13.3333%
Epoch: [20/20], Train Loss: 2.2554,rain Accuracy: 13.5000%, Test Loss: 1.7967, TestAccuracy: 15.0000%
Epoch: [20/20], Train Loss: 2.2554,rain Accuracy: 13.5000%, Test Loss: 2.2533, TestAccuracy: 16.0000%
Epoch: [30/30], Train Loss: 2.5176,rain Accuracy: 18.0000%, Test Loss: 0.3964, TestAccuracy: 20.0000%
Epoch: [30/30], Train Loss: 2.5176,rain Accuracy: 18.0000%, Test Loss: 0.8742, TestAccuracy: 15.0000%
Epoch: [30/30], Train Loss: 2.5176,rain Accuracy: 18.0000%, Test Loss: 1.3474, TestAccuracy: 13.3333%
Epoch: [30/30], Train Loss: 2.5176,rain Accuracy: 18.0000%, Test Loss: 1.8105, TestAccuracy: 12.5000%
Epoch: [30/30], Train Loss: 2.5176,rain Accuracy: 18.0000%, Test Loss: 2.2876, Tes